# Deep Direct Reinforcement Learning for Financial Signal Representation and Trading

## Abstract
資産取引において練達したトレーダーを打ち負かすことができるほどにコンピューターを訓練することはできるのか。この論文では再帰型深層ニューラルネットワークを

このモデルにおける取引＝市場の状況把握と最適行動の二つ逐次的意思決定からなる。  
従来手法と比べてこの動的な意思決定モデルは熟練したトレーダーの情報がない分だけ挑戦的。  
・・・状況を探索して各期の最適行動を決定する必要がある。  

自己学習と強化学習(RL)。
確率的最適制御と強化学習の研究。いくつかの事例では強化学習を備えたコンピューターが人間の能力を凌駕することもある。 
・・・トレーディングにおいてもRLが人間に勝つことができるのではないか？  
but トレーディングにおけるRLには従来のRLが対象にしてきた課題にはない問題がある。  

①金融環境の把握と表現の困難さから生じる問題。  
金融データには膨大な数のノイズ、ジャンプ、そして時系列を非定常なものに導くような運動が含まれている。  
移動平均やstochastic technical indicatorsが市場状態の評価のために用いられている。  
but drawback of technical analysis is its poor generalization ability.  

②トレーディングのダイナミックな意思決定から生じる問題。  
注文の発注は様々な要因を考慮に入れた上でのシステマテック的な仕事である。  
頻繁な取引ポジションの変更は利益を生まないだけでなくTransaction costやslippageから多大なロスを生む。  

この二つの問題に取り組むためにRDNN構造を、①の環境への反応問題と、②の意思決定問題に対処するために導入する。  
RDNN $\approx$ DNN for feature learning + RNN for market summarization  
※ To further improve the robustness for market summarization, the fuzzy learining concepts are introduced to reduce the uncertainty of the input data. 

## 3 Direct Deep Reinforcement Learning 

### A Direct Reinforcement Trading 
= 離散選択モデルで政策を決め動的計画法で解く。  
typical DRL is essentially a one layer RNN.  
$p_1, p_2, ..., p_t, ...$ : price sequences released from the exchange center.  
return : $ z_t = p_t - p_{t-1} $  
real-time trading decision : $ \delta_t \in \{long, neutral, short\} = \{1, 0, -1\} $  
profit : $ R_t = \delta_{t-1} z_t - c|\delta_t - \delta_{t-1}| $

profit = value function in each time point  
the accumulated value throughout the whole training period can ve defined as  
$$ max_\Theta U_T \{R_1...R_T|\Theta\} $$
簡単化のため、ここでは全体のvalue functionを各時点でのvalue functionの和として話を進める。  

the primary problem : how to solve it efficiently.  
従来の強化学習では離散空間に価値関数を設定してDPで繰り返し計算で解いていた。  
しかし、動的な取引の問題を、限られた数の離散空間で価値関数を学習させることで直接解くことは難しい。  
よって価値関数ではなく（本来価値関数があたえられた上でそれを最大化するように選ばれることで決まる）政策関数を学習する戦略を採用する。  
=== DRL  
a nonlinear function is adopted in DRL to approximate the trading action at each time point by
$$ \delta_t = tanh[<w, f_t> + b + u\delta_{t-1}] $$
f_t : feature vector of the current market condition at time t  
(w, b) are the coefficients for the feature regression  
つまり、回帰式にpenaltyを入れてtanhで表現している。bはbias項だがつまり1への回帰である。    

<img src="Fig1a.png">

In DRL, the recent m return values are directly adopted as the feature vector  
$$ f_t = [z_{t−m+1}, . . . , z_t] ∈ R^m. $$
In addition to the features, another term 
$$ uδ_{t−1} $$
is also added into the regression to take the latest trading decision into consideration.   
This term is used to discourage the agent to frequently change the trading positions and, hence, to avoid heavy TCs.  
With the linear transformation in the brackets, tanh(·) further maps the function into the range of (−1, 1) to approximate the final trading decision.   
The optimization of DRL aims to learn such a family of parameter set
$$ \Theta = \{w, u, b\} $$
that can maximize the global reward function in  
$$ max_\Theta U_T \{R_1...R_T|\Theta\} $$

### B Deep Recurrent Neural Network for DDR 

biasはfeatureに1を加えて回帰することで省略。  
DRLNNは自分の選択$\delta_t$を再帰的に利用。  
RNNを使うことは長期記憶を導入することであり過去の選択を継承できる。  
ただし、結局のところfeatureから市況の要約がうまくできない。回帰一般の問題。  

To implement feature learning, in this paper, we introduce the prevalent DL into DRL for simultaneously feature learning and dynamic trading.   
DL is a very powerful feature learning framework whose potentials have been extensively demonstrated in a number of machine learning problems.  
In detail, DL constructs a DNN to hierarchically transform the information from layer to layer.   
Such deep representation encourages much informative feature representations for a specific learning task.   
階層表現はより豊かに市況を表現できる（つまり非線形かつノンパラな手法）  

<img src="Fig1b.png">

上の青部分がDLの階層表現。  
By extending DL into DRL, the feature learning part(blue panel) is added to the RNN.  
DLを非線形な関数による近似と考えると、
$$ F_t = g_d(f_t) $$
であり、　　
$$ \delta_t = tanh[<w, F_t> + b + u\delta_{t-1}] $$
l+1層のそれぞれのノードはl層のすべてのノードと繋がっている。  
$a^l_i$をl層におけるi番目の入力とし、$o^l_i$を対応する出力とする。  
$$ a^l_i = <w^l_i, o^{(l-1)}> + b^l_i $$
$$ o^l_i = \frac{1}{1+e^{-a^l_i}} $$
標準シグモイド関数。  
$ o^{l-1}_i $はl-1層からの出力のベクトル。すべてのノードからの出力に重みが乗って入力される。  
4層の隠れ層に、それぞれ128ノード。  

### C Fuzzy Extensions to Reduce Uncertainties

The deep configuration well addresses the feature learning task in the RNN.   
↔︎ However, another important issue,   
#### i.e., data uncertainty in financial data, should also be carefully considered.  
Financial sequences contain high amount of unpredictable uncertainty due to the random gambling behind trading.  
Besides, a number of other factors, e.g., global economic atmosphere and some company rumors, may also affect the direction of the financial signal in real time.   
Therefore, reducing the uncertainties in the raw data is an important approach to increase the robustness for financial signal mining.  

In the artificial intelligence community, fuzzy learning is an ideal paradigm to reduce the uncertainty in the original data.  
Rather than adopting precise descriptions of some phenomena, fuzzy systems prefer to assign fuzzy linguist values to the input data.     
#### Such fuzzified representations can be easily obtained by comparing the real-world data with a number of fuzzy rough sets and then deriving the corresponding fuzzy membership degrees.   
#### Consequently, the learning system only works with these fuzzy representations to make robust control decisions.  

#### ラフ集合(wikipedia)
ラフ集合（ラフしゅうごう、Rough sets）とは上近似集合と下近似集合からなる集合で、非数値の対象を粗く（ラフに）記述することができるものである。これを用いることによって、他のデータマイニング手法からは得られにくい、非数値であったり矛盾のあるようなデータからの知識獲得が可能である。Rough sets theory（ラフ集合理論）の頭文字をとって RST や、Rough sets approach（ラフ集合アプローチ）の頭文字をとって RSA とも呼ばれる。
応用として、対象集合をファジィ集合に拡張したファジィ-ラフ集合理論 (Fuzzy-Rough sets theory) というものがある。

なるほど。ではfuzzy rough setsとは何か。  
For the financial problem discussed here, the fuzzy rough sets can be naturally defined according to the basicmovements of the stock price.   
In detail, the fuzzy sets are defined on the increasing, decreasing, and the no trend groups.  
The parameters in the fuzzy membership function can then be predefined according to the context of the discussed problem.  
Alternatively, they could be learned in a fully data-driven manner.   
The financial problem is highly complicated and it is hard to manually set up the fuzzy membership functions
Fig. 2.   

<img src="Fig2.png">

In [ ]:
Overview of fuzzy DRNNs for robust feature learning and self-taught
trading.
according to the experiences. Therefore, we prefer to directly
learn the membership functions and this idea will be detailed
in Section IV.
In fuzzy neural networks, the fuzzy representation part is
conventionally connected to the input vector ft (green nodes)
with different membership functions [35]. To note, in our
setting, we follow a pioneering work [35] to assign k different
fuzzy degrees to each dimension of the input vector. In the
cartoon of Fig. 2, only two fuzzy nodes (k = 2) are connected
to each input variable due to the space limitation. In our practical
implementation, k is fixed as 3 to describe the increasing,
decreasing, and no trend conditions. Mathematically, the
i th fuzzy membership function vi (·) : R → [0, 1] maps the
i th input as a fuzzy degree
o(l)
i = vi
$
a(l)
i
%
= e−
$
a(l)
i −mi
%2
/σ2
i &i. (7)
The Gaussian membership function with mean m and variance
σ2 is utilized in our system following the suggestions
of [37] and [38]. After getting the fuzzy representations, they
are directly connected to the deep transformation layer to seek
for the deep transformations.
In conclusion, the fuzzy DRNN (FDRNN) is composed of
three major parts as fuzzy representation, deep transformation,
and DRT.When viewing the FDRNN as a unified system, these
three parts, respectively, play the roles of data preprocessing
(reduce uncertainty), feature learning (deep transformation),
and trading policy making (RL). The whole optimization
framework is given as follows:
max
{",gd (·),v(·)}
UT (R1..RT )
s.t. Rt = δt−1zt − c|δt − δt−1|
δt = tanh(⟨w, Ft⟩+b + uδt−1)
Ft = gd(v(ft )) (8)
where there are three groups of parameters to be learned,
i.e., the trading parameters " = (w, b, u), fuzzy representations
v(·), and deep transformations gd (·). In the above
optimization, UT is the ultimate reward of the RL function,
δt is the policy approximated by the FRDNN, and Ft is
the high-level feature representat

## 4 DRNN LEARNING

$$ max_{\{\Theta, g_d(·), v(·)\}} U_T(R_1, R_T) $$
$$ s.t. R_t = \delta_{t-1}z_t - c|\delta_t - \delta_{t-1}) $$
$$ \delta_t = tanh(<w, F_t> + b + u\delta_{t-1}) $$
$$ F_t = g_d(v(f_t)) $$

はコンセプト的にはエレガントである。  
しかし、不幸にも相対的に最適化は困難でる。  
これは構成されたDNNが幾千もの隠れパラメーターを持ち、それらをinferしなければならないためである。  
このセクションでは、私たちは実践的な学習戦略を提示し、Aシステム初期化とBチューニングの2stepsでDNNをtrainする。

### A System Initializations

### B Task-Aware BPTT

<img src="Fig3.png">